# «Simple call seq» Report

Report for the "simple_call_seq" dataset (all included labels have at least 124 samples).
This test includes cnn 1d and 2d, lstm, each with image and hog features.

 - label extracting config: start/end with 10ms padding
 - hog config: ??
 - spectrogram config: sampling_rate=500000, width=100 window='Ham'

K-Fold with K=8 for 2762+406+124+162+149+148+591 samples, 8 bins of 15+15+15+15+15+15+15 and a remainder of 2642+286+4+42+29+28+471, for each fold: 4027 training, 105 validation, and 210 testing samples
 - val-bins: 1
 - test-bins: 2

In [ ]:
import glob
from IPython.display import display
from engine.settings import BSC_ROOT_DATA_FOLDER
from matplotlib import pyplot as plt
from utils.report import create_sorted_result

bin_size = 15
labels = ['B2', 'B3', 'B4', 'VS', 'VSV', 'UPS', 'none']
sorted_results = create_sorted_result('testing_scs_all2')[0:30]

## Results of the audio transforms (spectrograms)

20 randomly taken samples per label

In [ ]:
from utils.report import create_spectrogram_overview
image_paths = glob.glob(str(BSC_ROOT_DATA_FOLDER / 'simple_call_seq' / 'audio' / '*' /'sr500000_xpps2000_h300_wHam') + '/*.png')
create_spectrogram_overview(image_paths, labels, bin_size)

## Overview about the models

Kfold results over every model type

In [ ]:
from utils.report import create_model_overview
create_model_overview(sorted_results)

## Statistics

- Error-plots train + valid
- Confusion matrix per report per result

In [ ]:
from utils.report import create_statistics
create_statistics(sorted_results)

## identifies important pixels

Layer-Wise Relevance Propagation with bounded deep taylor

In [ ]:
import os
from utils.report import nn_cnn_2d_lrp

for result in sorted_results:
    recognizeBestAction = list(filter(lambda action: action['task'] == 'ParallelTask' and 'action' in action['props'] and action['props']['action'] == 'recognize_best_val_acc', result['pipeline']))
    model_path = recognizeBestAction[0]['props']['kwargs']['model_path']
    fig = nn_cnn_2d_lrp(os.path.dirname(model_path) + '/epoch_200.h5.json', 4)
    display(fig)
    fig.clear()
    plt.close(fig)
    break